# <center><font size="8">Prototipo de Oferta Dinámica Personalizada</font></center>

<div align="left"><font size="5">📌 Introducción</font></div>

## 1. Contexto y Objetivos

El presente proyecto se enfoca en el desarrollo de un sistema avanzado de **Oferta Dinámica (Dynamic Pricing)** con el objetivo primordial de **optimizar los ingresos y la rentabilidad** a través de la **personalización de tarifas**.  

En lugar de utilizar modelos de precio estáticos o ajustes basados únicamente en la demanda global, nuestro prototipo se cimenta en el **análisis individual del consumidor** para determinar el precio óptimo en tiempo real.  

Para lograr esta personalización, hemos diseñado un **pipeline de Machine Learning** que combina técnicas de **econometría**, **aprendizaje no supervisado** y **modelos predictivos**.

---

## 2. Metodología y Arquitectura del Modelo

La propuesta metodológica se basa en **tres fases interconectadas**, que transforman la sensibilidad del cliente en una tarifa personalizada:

---

### A. Estimación y Segmentación por Elasticidad (Fundamento Económico)

El primer pilar del prototipo fue la **estimación rigurosa de la elasticidad precio de la demanda ($\text{EPD}$)**.  
Esta métrica ha sido fundamental para entender el grado de respuesta de la cantidad demandada ante variaciones de precio en nuestros datos históricos.

Posteriormente, aplicamos un **algoritmo de clustering** (*aprendizaje no supervisado*) para **clasificar a los clientes** basándonos en su $\text{EPD}$ y otras características de comportamiento.  

Este *clustering* nos permitió crear **segmentos de clientes bien definidos** (por ejemplo: *“Extremadamente Elásticos”*, *“Inelásticos”*), estableciendo un **rango de ajuste de precios** (aumento o decremento) específico para cada grupo.

---

### B. Clasificación de Clientes Nuevos (Motor de XGBoost)

Una vez establecidos los segmentos de elasticidad, desarrollamos un **clasificador basado en el algoritmo XGBoost (eXtreme Gradient Boosting)**.  

La función de este modelo es crucial en la operativa:  
al recibir los datos de un nuevo cliente o una nueva interacción, **XGBoost predice a cuál de las categorías de elasticidad pre-clasificadas pertenece**.  

Esta predicción es el **factor determinante inicial** para estimar la tarifa personalizada, permitiéndonos saber si el sistema debe apuntar a un **aumento**, un **decremento** o un **precio base**.

---

### C. Determinación de la Oferta Dinámica (Red Neuronal)

Finalmente, para la etapa de determinación del precio, implementamos una **red neuronal artificial**.  

Este modelo de aprendizaje profundo **integra las características de la demanda**, la **clasificación de elasticidad obtenida de XGBoost**, y otras **variables dinámicas** (como inventario, hora del día y actividad de la competencia) para **estimar la oferta dinámica final y personalizada**.  

La red neuronal provee la **flexibilidad** y la **capacidad de capturar relaciones no lineales** necesarias para el ajuste preciso del precio a nivel individual.



<div align="left"><font size="5">📌 Objetivos</font></div>


- **1. Estimar la Elasticidad de la Demanda**  
  - Cuantificar la sensibilidad al precio ($\text{EPD}$) de los diferentes segmentos de clientes.  
  - Servirá como fundamento económico para la toma de decisiones de precio.

- **2. Segmentar Clientes por Comportamiento de Precio**  
  - Aplicar técnicas de *clustering para agrupar clientes en categorías de elasticidad homogéneas.  
  - Crear segmentos accionables que definan la estrategia inicial de aumento o decremento de precios.

- **3. Desarrollar un Motor de Clasificación de Clientes**  
  - Implementar y validar el algoritmo XGBoost para clasificar con alta precisión a nuevos clientes.  
  - Integrar la clasificación dentro de los segmentos de elasticidad previamente definidos.

- **4. Integrar un Pipeline Predictivo Completo**  
  - Crear un prototipo pipeline robusto que combine:  
    - **Clustering** (aprendizaje no supervisado)  
    - **Clasificación** (XGBoost)  
    - **Predicción** (Red Neuronal)  
- **5. Generar la tarifa final personalizada con tarifa dinámica**.



# <div align="left"><font size="5">📌 Librerias</font></div>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

from Tools import Tools4DataExtraction as T4DE
from Tools.Tools4Cluster import ClusteringData
from Tools.Tools4ClasSupervisada import ClusteringSupervisado
from Tools import Tools4TarifPer as T4TP
from Tools import Tools4Net as T4N 
from Tools import Tools4Elasticity as T4E

# <div align="left"><font size="5">📌Calculo de la elasticidad</font></div>

La Elasticidad Precio de la Demanda (EPD) mide cómo la cantidad demandada de un producto cambia en respuesta a una variación en su precio. Se calcula como 

$$\epsilon_{Q} = \frac{\Delta Q}{\Delta P}$$

### Clasificación de la Elasticidad de la Demanda según su valor

**Demanda Elástica** ($|EPD| > 1$):  
La cantidad demandada varía en una proporción mayor que el cambio en el precio.  
Un pequeño cambio en el precio provoca un cambio significativo en la cantidad demandada.  
*Ejemplo:* Bienes de lujo o productos con muchos sustitutos disponibles.

---

**Demanda Inelástica** ($|EPD| < 1$):  
La cantidad demandada varía en una proporción menor que el cambio en el precio.  
Un cambio en el precio tiene poco impacto en la cantidad demandada.  
*Ejemplo:* Bienes de primera necesidad o esenciales, como medicamentos o gasolina (a corto plazo).

---

**Elasticidad Unitaria** ($|EPD| = 1$):  
La cantidad demandada varía en la misma proporción que el cambio en el precio.

In [2]:
T4E.MainElas()

Memoria usada antes: 624.85 MB
Memoria usada después: 121.44 MB
Reducción: 80.6%
La elasticidad de la demanda es: -0.9054
El precio maximo a vender es: 967.6565


# <div align="left"><font size="5">📌Extracción de información del modelo de datos</font></div>

In [3]:
# Se extraen los datos del modelo de datos
D4NN, D4C, D4C1= T4DE.Get_Data()

Memoria usada antes: 624.85 MB
Memoria usada después: 121.44 MB
Reducción: 80.6%


# <div align="left"><font size="5">📌Clasificación no supervisada</font></div>

La clasificación de clientes permite entender cómo distintos grupos responden al precio y al servicio, habilitando estrategias de precio personalizado que maximizan ingresos y reducen pérdida de demanda. Se calcula el **precio óptimo** según la respuesta esperada del segmento (elasticidad estimada o probabilidad de compra).

### 🔹 Estrategias diferenciadas
- **Segmento elástico →** precios competitivos o descuentos.  
- **Segmento inelástico →** precios premium o aumentos graduales.  
- **Segmento leal →** estabilidad de precios.

---

## 📊 Beneficios Clave

- 💰 **Maximiza los ingresos** ajustando precios según la disposición a pagar.  
- 📉 **Reduce pérdida de demanda** por aumentos injustificados.  
- 🎯 **Mejora la precisión del modelo**, al trabajar con segmentos más homogéneos.  
- 🧠 **Permite estrategias personalizadas y accionables** según el tipo de cliente.


In [4]:
ClusteringData(False,D4C)

Modelo K-Means guardado exitosamente como: KmeansAllData.joblib


### Estrategia de Precios por *Cluster*

---

#### 1. Incrementar Precios (o Reducir Descuentos) 📈  
**Objetivo:** Maximizar el margen en clientes de alto valor.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **2** | Aumentar el precio. | **Élite:** Máximo gasto  y máxima actividad . Demanda inelástica. |
| **0** | Evaluar aumento moderado. | **Alto Valor:** Alto gasto. Monitorear para no perderlos por su menor actividad. |

---

#### 2. Bajar Precios (o Aumentar Promociones) 📉  
**Objetivo:** Impulsar el volumen y reactivar a clientes sensibles al precio o en riesgo.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **5** | Aumentar las promociones. | **Sensibles:** Bajo valor y alta indicación de sensibilidad a promociones. |
| **3** | Ofertas de reactivación. | **En Riesgo:** Gasto inicial alto, pero actividad posterior nula. Necesitan un incentivo fuerte para reengancharse. |

---

#### 3. Mantener Precios (Enfoque en Volumen y Venta Cruzada) 🛍️  
**Objetivo:** Fidelizar y aumentar la compra de productos adicionales sin tocar la base del precio.  

| **Cluster** | **Acción** | **Razón Clave** |
|--------------|-------------|-----------------|
| **1 y 4** | Mantener precios. | **Muy Activos:** Ya tienen una alta frecuencia de compra. La meta es venderles más productos, no bajarles ni bajarles el precio base. |


# <div align="left"><font size="5">📌Extracción de la base de datos de los clusters generados</font></div>

In [5]:
# se extrae la base de datos de los clusters generados
DBClus= T4DE.GetDB()

# <div align="left"><font size="5">📌Clasificación supervisada</font></div>


La implementación de un algoritmo de aprendizaje supervisado es fundamental para transformar una base de datos de clientes clasificados en un sistema de tarifa dinámica eficiente y rentable.

El valor agregado reside en pasar de la segmentación estática a la predicción continua. 

| Componente | Rol en el Modelo de Tarifa Dinámica | Beneficio Clave |
| :--- | :--- | :--- |
| **Clasificación Inicial de Clientes** | Sirve como *input* (una de las características) para el modelo de Machine Learning. | Proporciona una base de datos histórica y segmentada de alta calidad. |
| **Algoritmo de Aprendizaje Supervisado** | Entrena un modelo para predecir el comportamiento del cliente y calcular el precio óptimo basándose en datos históricos etiquetados (transacciones y resultados). | Maximiza el ingreso al ajustar la tarifa en tiempo real a la disposición a pagar individual. |
| **Red neuronal (Output)** | Permite al modelo predecir un valor continuo y exacto (el precio de venta) en lugar de limitarse a asignar categorías predefinidas. | Permite un ajuste fino y automatizado de precios, respondiendo a la demanda, inventario, precios de la competencia y el perfil específico del cliente simultáneamente. |



In [6]:
# Se aplica la clasificación supervisada a la base de datos de los clusters generados
ClusteringSupervisado(DBClus)

Modelo XGBoost guardado exitosamente como: modelo_xgboost_clientes.json


# <div align="left"><font size="5">📌Calculando la tarifa dinámica</font></div>

Una red neuronal artificial (RNA) es una herramienta poderosa para pronosticar la tarifa dinámica porque sobresale en el manejo de la complejidad y las relaciones no lineales inherentes a la fijación de precios en el mercado.

Su principal utilidad es su capacidad para modelar la elasticidad de la demanda de forma muy precisa, considerando simultáneamente una gran cantidad de variables.

In [7]:
T4N.ProcessingNet(D4NN)

df_= T4N.PrepareData4Fore(D4NN)
Fore= T4N.GetTodayData4Net(df_)
PrecioDin=T4N.NewClientsPredNet(Fore)

Epoch 1/2
6932/6932 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - loss: 17675.1816 - mae: 42.0969 - mse: 17675.1816 - val_loss: 130.9308 - val_mae: 7.8028 - val_mse: 130.9308
Epoch 2/2
6932/6932 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - loss: 120.6780 - mae: 7.5575 - mse: 120.6780 - val_loss: 110.7792 - val_mae: 6.9671 - val_mse: 110.7792
2167/2167 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step

El Error Absoluto Medio (MAE) final es de: 6.94 [Moneda]

La Raíz del Error Cuadrático Medio (RMSE) final es de: 10.44 [Moneda]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step


In [16]:
df_['PRECIO DINAMICO']=PrecioDin
TodayDataPD= df_.copy()

# <div align="left"><font size="5">📌Obteniendo la elasticidad calculada</font></div>

La elasticidad calculada se guarda en un modelo de datos para posteriormente cargarla y hacer los cálculos necesarios en la segmentación de los clientes.

In [8]:
DataElas= T4E.GetDataElasticity()
Elas=DataElas['Elasticidad']

# <div align="left"><font size="5">📌Obteniendo datos del presente día para la tarifa personalizada</font></div>

Se cargan los datos del día presente del modelo de datos

In [9]:
# Se obtienen los datos de los clientes para obtener la tarifa personalizada
TodayData4C= T4TP.GetTodayData4Cluster(D4C1)

# <div align="left"><font size="5">📌Calculando el porcentaje de aumento para la tarifa personalizada</font></div>

Ya teniendo entrenados los algoritmos que clasificaron la data histórica de clientes, de manera supervisada y no supervisada, se procede a realizar una prueba de calidad con los datos del día corriente para poder observar el comportamiento de los precios personalizados.

In [10]:
# se simula la obtencion del cluster del cliente que está comprando
# para ver que tarifa personalizada se le asigna

import pandas as pd

# 1. Inicializa una lista vacía para guardar los resultados
lista_resultados = []

# 2. Itera y agrega cada DataFrame a la lista
for row in range(len(TodayData4C)):
    # 1. Crea la fila (DataFrame de 1xN)
    InfoClient = pd.DataFrame(TodayData4C.iloc[row]).T
    
    # 2. Procesa la información (asumo que T4TP.GetCluster devuelve Cluster y desc)
    Cluster, desc = T4TP.GetCluster(InfoClient, DBClus,Elas)
    
    # 3. Agrega la nueva columna
    InfoClient["%_Tarifa Personalizada"] = desc
    InfoClient["Cluster"] = Cluster
    
    # 4. Agrega el DataFrame resultante a la lista
    lista_resultados.append(InfoClient)

# 3. Concatena todos los DataFrames de la lista de una sola vez
TodayDataTP = pd.concat(lista_resultados, ignore_index=True)

# El DataFrame 'final' ahora contiene todas las filas concatenadas

# <div align="left"><font size="5">📌Merge de las tarifas</font></div>

Para finalizar este pipeline, se combinan los dataframes obtenidos de la red neuronal del precio dinámico con el del algoritmo de clasificación del precio personalizado

In [17]:
df_combinado = pd.merge(
    TodayDataPD,
    TodayDataTP,
    on='NOMBRE_PASAJERO',  # La columna clave que comparten
    how='inner'    # El tipo de unión que deseas
)

In [18]:
df_PD= df_combinado[['NOMBRE_PASAJERO', 'EMAIL','Cluster', 'PRECIO DINAMICO', '%_Tarifa Personalizada']]
df_PD ['Precio final con IVA']= df_PD['PRECIO DINAMICO']*(df_PD['%_Tarifa Personalizada']/100)+ df_PD['PRECIO DINAMICO']+df_PD['PRECIO DINAMICO']*0.15
df_PD ['Precio final con IVA']= np.round(df_PD ['Precio final con IVA'])

El siguiente dataframe muestra los precios dinámicos con el 15 porciento de IVA para cada uno de los usuarios del presente día según su clasificación obtenida con los algoritmos de aprendizaje. 

In [19]:
df_PD

NOMBRE_PASAJERO  \
0                   BRAUO9LIO MACIAS   
1                  XIMENA BALDERRAMA   
2               ALEXIS EDUARDO ANAYA   
3       EDUARDO JONATHAN CORTES LARA   
4                      NICOLAS RUBIO   
5                 LUIS FELIPE CASTRO   
6                   JUSSIEL TRINIDAD   
7                    SAGRARIO ENCISO   
8                       SERGIO SORIA   
9                VANIA GALAZA CURIEL   
10                 FERNANDO BAUTISTA   
11           FREDY CASTELAN GONXALEZ   
12       ALAN SAID SANDOVAL VALENCIA   
13         ALEJANDRO RAMIREZ SOLARES   
14            MANUEL CORTÉS SANDOVAL   
15               ANDREA MEDINA ROSAS   
16                   HORACIO JIMENEZ   
17                 GASBRIEL MAGALLON   
18  FALU FRANCISCO HERNANDEZ GARRIDO   
19    MARIA DOLORES BUSTOS HERNANDEZ   
20                        JULIA CRUZ   
21        ALFREDO ISMAEL ROMÁN MASSO   
22           DIEGO EDGAR RIOS TORRES   
23            PETRA CARINO CASTREJON   
24                LORENO MONTES RIOS   
25          ISAURA DOMINGUES HERRERA   

                                           EMAIL  Cluster  PRECIO DINAMICO  \
0                   brauo9lio.macias@ejemplo.com        1       965.287048   
1                  ximena.balderrama@ejemplo.com        1       964.002197   
2               alexis.eduardo.anaya@ejemplo.com        1       963.635925   
3       eduardo.jonathan.cortes.lara@ejemplo.com        1       964.246399   
4                      nicolas.rubio@ejemplo.com        1       964.749939   
5                 luis.felipe.castro@ejemplo.com        1       964.684204   
6                   jussiel.trinidad@ejemplo.com        1       962.523010   
7                    sagrario.enciso@ejemplo.com        1       962.374695   
8                       sergio.soria@ejemplo.com        1       964.304504   
9                vania.galaza.curiel@ejemplo.com        1       962.671448   
10                 fernando.bautista@ejemplo.com        1       962.077698   
11           fredy.castelan.gonxalez@ejemplo.com        5       967.497375   
12       alan.said.sandoval.valencia@ejemplo.com        5       967.070068   
13         alejandro.ramirez.solares@ejemplo.com        5       967.314270   
14            manuel.cortés.sandoval@ejemplo.com        5       965.820496   
15               andrea.medina.rosas@ejemplo.com        1       961.336060   
16                   horacio.jimenez@ejemplo.com        5       966.452393   
17                 gasbriel.magallon@ejemplo.com        1       964.066833   
18  falu.francisco.hernandez.garrido@ejemplo.com        1       962.784729   
19    maria.dolores.bustos.hernandez@ejemplo.com        3       961.286316   
20                        julia.cruz@ejemplo.com        1       961.137939   
21        alfredo.ismael.román.masso@ejemplo.com        1       963.513184   
22           diego.edgar.rios.torres@ejemplo.com        1       963.661560   
23            petra.carino.castrejon@ejemplo.com        1       960.544128   
24                loreno.montes.rios@ejemplo.com        3       962.325500   
25          isaura.domingues.herrera@ejemplo.com        1       958.932983   

    %_Tarifa Personalizada  Precio final con IVA  
0                 0.000000                1110.0  
1                 0.000000                1109.0  
2                 0.000000                1108.0  
3                 0.000000                1109.0  
4                 0.000000                1109.0  
5                 0.000000                1109.0  
6                 0.000000                1107.0  
7                 0.000000                1107.0  
8                 0.000000                1109.0  
9                 0.000000                1107.0  
10                0.000000                1106.0  
11               -4.527081                1069.0  
12               -4.527081                1068.0  
13               -4.527081                1069.0  
14               -4.527081                1067.0  
15               

In [ ]:
df_PD.to_csv('datos_dinamicos.csv', index=False)